## Analyze news release of pharmaceutical co.

In [2]:
with open("sec/data/ABBV/2012-12-20-0001104659-12-085571.txt") as fin:
    for line in fin:
       print line     

<SEC-DOCUMENT>0001104659-12-085571.txt : 20121220

<SEC-HEADER>0001104659-12-085571.hdr.sgml : 20121220

<ACCEPTANCE-DATETIME>20121220171203

ACCESSION NUMBER:		0001104659-12-085571

CONFORMED SUBMISSION TYPE:	8-K

PUBLIC DOCUMENT COUNT:		3

CONFORMED PERIOD OF REPORT:	20121214

ITEM INFORMATION:		Changes in Registrant's Certifying Accountant

ITEM INFORMATION:		Financial Statements and Exhibits

FILED AS OF DATE:		20121220

DATE AS OF CHANGE:		20121220



FILER:



	COMPANY DATA:	

		COMPANY CONFORMED NAME:			AbbVie Inc.

		CENTRAL INDEX KEY:			0001551152

		STANDARD INDUSTRIAL CLASSIFICATION:	PHARMACEUTICAL PREPARATIONS [2834]

		IRS NUMBER:				320375147

		STATE OF INCORPORATION:			DE

		FISCAL YEAR END:			1231



	FILING VALUES:

		FORM TYPE:		8-K

		SEC ACT:		1934 Act

		SEC FILE NUMBER:	001-35565

		FILM NUMBER:		121278004



	BUSINESS ADDRESS:	

		STREET 1:		1 NORTH WAUKEGAN ROAD

		CITY:			NORTH CHICAGO

		STATE:			IL

		ZIP:			60064

		BUSINESS PHONE:		(847) 937-6100



	MAIL 

In [3]:
# get the content between <TEXT> and </TEXT>; returns the rest too
def get_text_and_rest(txt):
    idx_TEXT = txt.find("<TEXT>")
    if idx_TEXT == -1:
        return None, None
    else:
        idx_slash_TEXT = txt.find("</TEXT>", idx_TEXT + 6)
        if idx_slash_TEXT == -1:
            return txt[idx_TEXT + 6:], None
        else:
            return txt[idx_TEXT +6:idx_slash_TEXT], txt[idx_slash_TEXT + 7:]

In [5]:
# Read all the content between <TEXT> and </TEXT>. Can't assume legal XML format
from bs4 import BeautifulSoup
with open("sec/data/ABBV/2012-12-20-0001104659-12-085571.txt") as fin:
     text = fin.read()

first, rest = get_text_and_rest(text)
if first:
    soup = BeautifulSoup(first, "html.parser")
    print(soup.get_text())
while rest:
    first, rest = get_text_and_rest(rest)
    if first:
        soup = BeautifulSoup(first, "html.parser")
        print(soup.get_text())
    rest = rest









 

 
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D. C. 20549
 

 
FORM 8-K
 
CURRENT REPORT
Pursuant to Section 13 or 15(d) of the
Securities Exchange Act of 1934
 
December 14, 2012
Date of Report (Date of earliest event reported)
 
ABBVIE INC.
(Exact name of registrant as specified in its charter)
 

 




Delaware 

  

001-35565 

  

32-0375147  


(State or other Jurisdiction 

  

(Commission File Number) 

  

(IRS Employer  


of Incorporation) 

  

  

  

Identification No.)   

 

 
1 North Waukegan Road
North Chicago, Illinois 60064-6400
(Address of principal executive offices)(Zip Code)
 
Registrants telephone number, including area code:  (847) 932-7900
 
Check the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions:
 
o            Written communications pursuant to Rule 425 under the Securities Act (17 CFR 230.425)
 
o            Solicit

### Create a function for extracting text from a sec filing text file

In [6]:
from bs4 import BeautifulSoup
# input: the string content of a sec filing file
# output: the text extracted from the content between <TEXT> and </TEXT> by stripping off all the HTML tags
def extract_text(htmlstring):
    ans = ""
    first, rest = get_text_and_rest(htmlstring)
    if first:
        soup = BeautifulSoup(first, "html.parser")
        ans = ans + soup.get_text()
    while rest:
        first, rest = get_text_and_rest(rest)
        if first:
            soup = BeautifulSoup(first, "html.parser")
            ans = ans + soup.get_text()
    return ans

In [7]:
with open("sec/data/ABBV/2012-12-20-0001104659-12-085571.txt") as fin:
    text = fin.read()
    print extract_text(text) 








 

 
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D. C. 20549
 

 
FORM 8-K
 
CURRENT REPORT
Pursuant to Section 13 or 15(d) of the
Securities Exchange Act of 1934
 
December 14, 2012
Date of Report (Date of earliest event reported)
 
ABBVIE INC.
(Exact name of registrant as specified in its charter)
 

 




Delaware 

  

001-35565 

  

32-0375147  


(State or other Jurisdiction 

  

(Commission File Number) 

  

(IRS Employer  


of Incorporation) 

  

  

  

Identification No.)   

 

 
1 North Waukegan Road
North Chicago, Illinois 60064-6400
(Address of principal executive offices)(Zip Code)
 
Registrants telephone number, including area code:  (847) 932-7900
 
Check the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions:
 
o            Written communications pursuant to Rule 425 under the Securities Act (17 CFR 230.425)
 
o            Solicit

In [11]:
# write a file to the disk
import os
def write_file(filename, file_txt):
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    with open(filename, "w") as f:
        f.write(file_txt)
                    
    
data_dir_path = "sec/data/"
filtered_data_path = "sec/filtered_data/"
count = 0
for dir_name in os.listdir(data_dir_path):
    if os.path.isdir(data_dir_path + dir_name):
        for file_name in os.listdir(data_dir_path + dir_name):
            try:
                file_path = data_dir_path + dir_name + "/" + file_name
                count = count + 1
                #print(file_path)
                with open(file_path) as fin:
                    html_text = fin.read()
                    content_text = extract_text(html_text)
                    if content_text.lower().find("clinical trial") != -1 and content_text.lower().find("press release") != -1:
                        file_path = filtered_data_path + "positive" + "/" + dir_name + "/" + file_name
                        write_file(file_path, content_text.encode('utf8'))
                    else:
                        file_path = filtered_data_path + "negative" + "/" + dir_name + "/" + file_name
                        write_file(file_path, content_text.encode('utf8'))
            except:
                print("Can't process the file: " + file_name)
                
print("Total Number of Files: " + str(count))


Can't process the file: 2010-02-11-0001193125-10-028617.txt
Can't process the file: 2010-10-15-0001193125-10-230113.txt
Can't process the file: 2010-07-14-0001193125-10-158844.txt
Can't process the file: 2010-01-13-0001144204-10-001748.txt
Can't process the file: 2010-04-13-0001193125-10-082036.txt
Can't process the file: 2010-11-02-0000930413-10-005371.txt
Can't process the file: 2010-01-11-0000950123-10-001696.txt
Can't process the file: 2010-12-30-0000950123-10-117774.txt
Can't process the file: 2010-01-12-0000950123-10-001923.txt
Can't process the file: 2010-08-04-0001193125-10-177257.txt
Can't process the file: 2010-09-21-0001144204-10-050254.txt
Can't process the file: 2010-11-16-0001144204-10-061218.txt
Can't process the file: 2010-08-03-0001193125-10-174388.txt
Can't process the file: 2010-02-08-0001144204-10-005941.txt
Can't process the file: 2010-09-14-0000950123-10-086085.txt
Can't process the file: 2010-05-05-0001193125-10-107607.txt
Can't process the file: 2010-04-12-00011